In [ ]:
# Convert list of steam IDs into API "batch" format

def ids_list_to_batch(platform, ids):
    batch = [{"platformId":str(platform), "uniqueId":str(ids[i])} for i in range(len(ids))]
    
    return batch
# Send request with batch of steam IDs and return a dict of player stats
# Request up to 10 steam IDs at a time

def batch_to_dict(auth, batch):
    url = "https://api.rocketleaguestats.com/v1/player/batch"
    headers = {"Authorization":auth}
    
    response = requests.post(url, headers=headers, json=batch)
    try:
        stats_list = ast.literal_eval(response.text)
    except:
        stats_list = response.text
        
    return stats_list
def append_record(record):
    with open("../data/db.txt", "a") as f:
        json.dump(record, f)
        f.write(os.linesep)
# Take list of steam IDs, send requests to RLS API in batches,
# create a list of dict responses, return the list
# NOTE: Only 2 requests allowed per second
# NOTE: Server is super slow so I don't think 2/s matters...

def create_db(steam_ids, auth, db=[]):
    full_batches = len(steam_ids) // 10
    
    for i in tqdm(range(full_batches)):
        start = i * 10
        stop = i * 10 + 10
        
        batch = ids_list_to_batch(1, steam_ids[start:stop])
        stats_dicts = batch_to_dict(auth, batch)
        
        for i in range(len(stats_dicts)):
            append_record(stats_dicts[i])
            
    batch = ids_list_to_batch(1, steam_ids[stop:])
    stats_dict = batch_to_dict(auth, batch)
    
    for i in range(len(stats_dicts)):
        append_record(stats_dicts[i])
    
    return 0
